In [18]:
import numpy as np
import pandas as pd

import zipfile


from theano import tensor

import fuel
import h5py
from fuel.datasets.hdf5 import H5PYDataset
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten
from fuel.datasets.text import TextFile

from blocks.bricks import Linear, Logistic, Softmax
from blocks.bricks.cost import CategoricalCrossEntropy
from blocks.roles import WEIGHT
from blocks.graph import ComputationGraph
from blocks.filter import VariableFilter
from blocks.initialization import IsotropicGaussian, Constant
from blocks.algorithms import GradientDescent, Scale, AdaGrad, Adam
from blocks.extensions.monitoring import DataStreamMonitoring
from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks_extras.extensions.plot import Plot

%matplotlib inline

In [19]:
filename = "./data/text8.zip"

def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return f.read(name).split()
  f.close()
  
#words = read_data(filename)
#print 'Data size', len(words)

In [20]:
from fuel.datasets.base import Dataset

class WordWindow(Dataset):
    def __init__(self, **kwargs):
        self.provides_sources = ('features', 'targets')
         # for technical reasons
        self.axis_labels = None
        with self.open() as fh:
            self.corpus = fh.read().split()
        #print self.corpus 
        self.vocabulary_size = len(set(self.corpus))
        self.vocabulary = pd.factorize(self.corpus)
        self.num_instances = len([((self.vocabulary[0][i],self.vocabulary[0][i+2]),self.vocabulary[0][i+1]) for i in xrange(len(self.corpus)-2)])
        super(WordWindow, self).__init__(**kwargs)
    
    def get_data(self, state=None, request=None):
        data = [((self.vocabulary[0][i],self.vocabulary[0][i+2]),self.vocabulary[0][i+1]) for i in xrange(len(self.corpus)-2)]
        return data

    def open(self):
        return open('./data/small')
    
    def close(self,fh):
        fh.close()
    
dataset = WordWindow()
print dataset.vocabulary_size
print dataset.vocabulary

7
(array([0, 1, 2, 0, 3, 0, 3, 2, 0, 1, 0, 4, 5, 0, 1, 0, 3, 5, 0, 4, 0, 4, 6,
       0, 1, 0, 4, 6, 0, 3]), array(['the', 'king', 'loves', 'queen', 'dwarf', 'hates', 'poisons'], dtype=object))


In [21]:
from fuel.streams import DataStream
#for data in DataStream(dataset).get_epoch_iterator():
#    print data

dataset.get_data()

[((0, 2), 1),
 ((1, 0), 2),
 ((2, 3), 0),
 ((0, 0), 3),
 ((3, 3), 0),
 ((0, 2), 3),
 ((3, 0), 2),
 ((2, 1), 0),
 ((0, 0), 1),
 ((1, 4), 0),
 ((0, 5), 4),
 ((4, 0), 5),
 ((5, 1), 0),
 ((0, 0), 1),
 ((1, 3), 0),
 ((0, 5), 3),
 ((3, 0), 5),
 ((5, 4), 0),
 ((0, 0), 4),
 ((4, 4), 0),
 ((0, 6), 4),
 ((4, 0), 6),
 ((6, 1), 0),
 ((0, 0), 1),
 ((1, 4), 0),
 ((0, 6), 4),
 ((4, 0), 6),
 ((6, 3), 0)]

In [29]:
from blocks.bricks.lookup import LookupTable
from blocks.initialization import IsotropicGaussian, Constant


VOCAB_DIM = dataset.vocabulary_size
EMBEDDING_DIM = min(5,VOCAB_DIM)
CONTEXT = 1

def makeGraph():
    Xs = tensor.lmatrix("features")
    y = tensor.matrix('targets')
        
    w1 = LookupTable(name="w1", length=VOCAB_DIM, dim=EMBEDDING_DIM)
    w2 = Linear(name='w2', input_dim=EMBEDDING_DIM, output_dim=VOCAB_DIM)

    hidden = tensor.mean(w1.apply(Xs), axis=1)
    y_hat = Softmax().apply(w2.apply(hidden))
    
    w1.weights_init = w2.weights_init = IsotropicGaussian(0.01)
    w1.biases_init = w2.biases_init = Constant(0)
    w1.initialize()
    w2.initialize()
    
    cost = CategoricalCrossEntropy().apply(y, y_hat)
    
    cg = ComputationGraph(cost)
    W1, W2 = VariableFilter(roles=[WEIGHT])(cg.variables)
    
    #cost = cost + 0.005 * (W1 ** 2).sum() + 0.005 * (W2 ** 2).sum()

    
    
    return cg,(W1,W2),cost

In [32]:
cg, (W1, W2), cost = makeGraph()
main = MainLoop(data_stream = DataStream(
                    dataset,
                    iteration_scheme=SequentialScheme(dataset.num_instances, batch_size=10)),
                algorithm = GradientDescent(
                    cost = cost, 
                    parameters = cg.parameters,
                    step_rule = Scale(learning_rate=0.1)),
                extensions = [
                    #ProgressBar(),
                    FinishAfter(after_n_epochs=1),
                    Printing(),
                    #TrainingDataMonitoring(variables=[cost]),
])

main.run()


-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:



ValueError: ('Bad input argument to theano function with name "/home/ray/.local/bin/anaconda2/lib/python2.7/site-packages/blocks/algorithms/__init__.py:261"  at index 0(0-based), setting an array element with a sequence.\n\nOriginal exception:\n\tValueError: Bad input argument to theano function with name "/home/ray/.local/bin/anaconda2/lib/python2.7/site-packages/blocks/algorithms/__init__.py:261"  at index 0(0-based), setting an array element with a sequence.', 'setting an array element with a sequence.')